<a href="https://colab.research.google.com/github/aimerou/deep-learning/blob/main/notebooks/keras_hyperopt_optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install the PyDrive wrapper & import libraries.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Copy/download the file
fid = drive.ListFile({'q':"title='keras_hyperopt_optimization.ipynb'"}).GetList()[0]['id']
f = drive.CreateFile({'id': fid})
f.GetContentFile('keras_hyperopt_optimization.ipynb')

In [ ]:
!pip install hyperopt
!pip install hyperas
!pip install tensorflow

In [3]:
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical

In [4]:
def data():
    """
    Data providing function:

    This function is separated from model() so that hyperopt
    won't reload data for each evaluation run.
    """
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data(path="mnist.npz")
    x_train = x_train.reshape(x_train.shape[0], 784)
    x_test = x_test.reshape(x_test.shape[0], 784)
    y_train = to_categorical(y_train)
    y_test = to_categorical(y_test)
    return x_train, y_train, x_test, y_test

In [5]:
def model(x_train, y_train, x_test, y_test):
    """
    Model providing function:

    Create Keras model with double curly brackets dropped-in as needed.
    Return value has to be a valid python dictionary with two customary keys:
        - loss: Specify a numeric evaluation metric to be minimized
        - status: Just use STATUS_OK and see hyperopt documentation if not feasible
    The last one is optional, though recommended, namely:
        - model: specify the model just created so that we can later use it again.
    """
    model_mlp = Sequential()
    model_mlp.add(Dense({{choice([32, 64, 128, 256, 512])}}, activation='relu', input_shape= (784,)))
    model_mlp.add(Dense(10, activation='softmax'))
    model_mlp.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')
    
    model_mlp.fit(x_train, y_train,
          epochs=50,
          verbose=False,
          validation_data=(x_test, y_test))
    score, acc = model_mlp.evaluate(x_test, y_test, verbose=True)
    print('Test accuracy:', acc)
    return {'loss': -acc, 'status': STATUS_OK, 'model': model_mlp}

In [ ]:
best_run, best_model = optim.minimize(model=model,
                                      data=data,
                                      algo=tpe.suggest,
                                      max_evals=2,
                                      notebook_name='hyperopt',
                                      trials=Trials())

In [8]:
X_train, Y_train, X_test, Y_test = data()
print("Evalutation of best performing model:")
eval = best_model.evaluate(X_test, Y_test)
print(f"loss : {eval[0]}, accuracy : {eval[1]}")
print(f"Best performing model chosen hyper-parameters : {best_run}")

Evalutation of best performing model:
313/313 [==============================] - 1s 2ms/step - loss: 1.1357 - accuracy: 0.9620
loss : 1.135739803314209, accuracy : 0.9620000123977661
Best performing model chosen hyper-parameters : {'Dense': 4}
